In [ ]:
# Optional: uncomment to install missing libs in the notebook environment
import sys
# !{sys.executable} -m pip install MetaTrader5 stable-baselines3 gymnasium numpy pandas matplotlib joblib


In [2]:
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

# Paths (based on your earlier config)
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")

MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
METRICS_FILE = os.path.join(MODEL_DIR, "eval_metrics.json")

# Runtime config
WINDOW = 50
TF_MT5 = mt5.TIMEFRAME_M1
DRY_RUN = True               # KEEP True while testing
DEFAULT_LOT = 0.1
MAX_POS_PER_SYMBOL = 2

# SL/TP: SL derived from volatility estimate; TP = TP_MULT * SL
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3

# Trailing SL pips (will use half of SL estimation as applied)
TRAIL_PIPS = 5

# Logging
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [ ]:
# ================================================================
# ⚙️ Cell 4 - Symbols Selection
# ================================================================
SYMBOLS = [
    # --- Volatility Indices (Standard) ---
    "Volatility 10 Index",
    "Volatility 25 Index",
    "Volatility 50 Index",
    "Volatility 75 Index",
    "Volatility 100 Index",

    # --- Volatility 1s Indices ---
    "Volatility 10 (1s) Index",
    "Volatility 25 (1s) Index",
    "Volatility 50 (1s) Index",
    "Volatility 75 (1s) Index",
    "Volatility 100 (1s) Index",

    # --- Volatility 10s Indices ---
    "Volatility 10 (10s) Index",
    "Volatility 25 (10s) Index",
    "Volatility 50 (10s) Index",
    "Volatility 75 (10s) Index",
    "Volatility 100 (10s) Index",

    # --- Jump Indices ---
    "Jump 10 Index",
    "Jump 25 Index",
    "Jump 50 Index",
    "Jump 75 Index",
    "Jump 100 Index",

    # --- Step Indices ---
    "Step Index 25",
    "Step Index 50",
    "Step Index 75",
    "Step Index 100",

    # --- Forex Reference ---
    "EURUSD"
]

In [ ]:
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def raw_from_safe(safe: str) -> str:
    return safe.replace("_", " ")

def safe_from_raw(raw: str) -> str:
    return make_safe_name(raw)


In [ ]:
# automatic mapping - start with likely mappings; you can expand/update this dict
MANUAL_SYMBOL_MAP = {
    "EURUSD": "EURUSD",
    "Jump_100_Index": "JUMP100",
    "Jump_10_Index": "JUMP10",
    "Jump_25_Index": "JUMP25",
    "Jump_50_Index": "JUMP50",
    "Jump_75_Index": "JUMP75",
    "Volatility_100_1s_Index": "VOL100_1S",
    "Volatility_100_Index": "VOL100",
    "Volatility_10_1s_Index": "VOL10_1S",
    "Volatility_10_Index": "VOL10",
    "Volatility_25_1s_Index": "VOL25_1S",
    "Volatility_25_Index": "VOL25",
    "Volatility_50_1s_Index": "VOL50_1S",
    "Volatility_50_Index": "VOL50",
    "Volatility_75_1s_Index": "VOL75_1S",
    "Volatility_75_Index": "VOL75",
    # Add more if needed
}

def get_mt5_symbol(safe_name: str) -> str:
    """Return broker symbol name for a safe_name using MANUAL_SYMBOL_MAP or fallback to spaces."""
    return MANUAL_SYMBOL_MAP.get(safe_name, raw_from_safe(safe_name))

def fetch_data_for_symbol(safe_name: str, window: int = WINDOW):
    """
    Fetch recent bars from MT5 for the broker symbol mapped from safe_name.
    Returns DataFrame with columns open,high,low,close,volume,Close_raw
    """
    raw_symbol = get_mt5_symbol(safe_name)
    info = mt5.symbol_info(raw_symbol)
    if info is None:
        print(f"❌ Symbol not found in MT5: {raw_symbol}")
        return None
    # ensure visible
    if not info.visible:
        try:
            mt5.symbol_select(raw_symbol, True)
        except Exception:
            pass
    count = window + 60  # a buffer
    bars = mt5.copy_rates_from_pos(raw_symbol, TF_MT5, 0, count)
    if bars is None or len(bars) < window + 2:
        print(f"❌ Not enough bars for {raw_symbol} (got {0 if bars is None else len(bars)})")
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume': 'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df


In [ ]:
# scalers: read per-asset CSVs created earlier (mean/std)
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                print("Skipping", p, "- unexpected format")
                continue
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found:", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch")
    return emb_dict

# Load them
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded datasets:", len(datasets), "scalers:", len(scalers), "embeddings:", len(embeddings))
safe_list = sorted(datasets.keys())
print("Assets:", safe_list)


In [1]:
# Load PPO and VecNormalize
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO model:", MODEL_FILE)

vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE)
        print("Loaded VecNormalize:", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize load failed:", e)
        vecnorm = None


NameError: name 'os' is not defined

In [ ]:
def fetch_and_build_obs(safe_name, window, scalers, embeddings, datasets, safe_list):
    """
    Build observation consistent with training:
    obs shape = (window, 5 + 1 + embed_dim)
    where 5 = o_pc,h_pc,l_pc,c_pc,v_pc ; 1 = balance (fixed 1.0) ; embed_dim from embeddings.
    """
    # prefer live bars, but fallback to preloaded dataset if fetch fails
    df_live = fetch_data_for_symbol(safe_name, window)
    if df_live is None:
        # fallback to prepared dataset
        if safe_name not in datasets:
            print("No data available for", safe_name)
            return None, None, None
        df = datasets[safe_name].iloc[-(window+1):].copy()
        # reconstruct OHLCV in price-space from Close_raw if needed is not possible; but we expect dataset to contain pct-change already
        # We'll use stored pct columns if present:
        df_pct = df[['o_pc','h_pc','l_pc','c_pc','v_pc']].copy()
        last_price = float(df['Close_raw'].iloc[-1])
    else:
        # compute pct-change from live OHLCV
        df = df_live
        df_pct = df[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            print("Not enough pct rows for", safe_name)
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df['Close_raw'].iloc[-1])

    # when using infile dataset, df_pct above already length==window
    if df_pct.shape[0] < window:
        print("Window mismatch for", safe_name)
        return None, None, None

    # get scaler
    if safe_name not in scalers:
        print("No scaler for", safe_name)
        return None, None, None
    s = scalers[safe_name]
    mean = np.array(s['mean'].values if hasattr(s['mean'], 'values') else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], 'values') else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)

    # Ensure df_pct column order matches mean/std indices: scalers were saved with index names o_pc,h_pc,l_pc,c_pc,v_pc
    cols = ['o_pc','h_pc','l_pc','c_pc','v_pc']
    # If df_pct currently has original OHLCV names, convert order
    if list(df_pct.columns) != cols:
        # attempt to rename if current names are open,high,...
        if all(c in df_pct.columns for c in ['open','high','low','close','volume']):
            df_pct = df_pct[['open','high','low','close','volume']]
        else:
            # assume df_pct already has o_pc,h_pc...
            pass

    # get features array of shape (window,5)
    features = df_pct[cols].values.astype(np.float32) if all(c in df_pct.columns for c in cols) else df_pct.values.astype(np.float32)
    features_scaled = (features - mean) / std

    # embedding
    if safe_name in embeddings:
        emb_vec = np.array(embeddings[safe_name], dtype=np.float32)
    else:
        emb_dim = next(iter(embeddings.values())).shape[0] if len(embeddings)>0 else 8
        emb_vec = np.zeros(emb_dim, dtype=np.float32)
    emb_block = np.repeat(emb_vec.reshape(1,-1), window, axis=0)

    # balance column (used in training) - keep as ones
    balance_col = np.ones((window,1), dtype=np.float32)

    obs = np.concatenate([features_scaled, balance_col, emb_block], axis=1).astype(np.float32)

    # If VecNormalize exists and matches expected flattened size, attempt to apply (vecnorm expects flattened obs in older SB3 versions)
    if vecnorm is not None:
        try:
            flat = obs.reshape(1, -1)
            mean_rms = getattr(vecnorm.obs_rms, "mean", None)
            var_rms = getattr(vecnorm.obs_rms, "var", None)
            if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                obs = flat_norm.reshape(obs.shape).astype(np.float32)
            else:
                # some SB3 versions provide normalize_obs
                try:
                    obs = vecnorm.normalize_obs(obs, False)
                except Exception:
                    pass
        except Exception as e:
            print("VecNormalize apply error:", e)

    # volatility estimate for SL/lot sizing
    vol_est = float(df_pct['c_pc'].std()) if 'c_pc' in df_pct.columns else float(np.std(features[:,3]))

    return obs, vol_est, last_price


In [1]:
def pip_value(symbol):
    return 0.01 if "JPY" in symbol.upper() else 0.0001

def estimate_sl_pips_from_vol(raw_symbol, last_price, vol_est, min_pips=5, max_pips=200):
    if "JPY" in raw_symbol.upper():
        pip = 0.01
    else:
        pip = 0.0001
    abs_move = vol_est * last_price
    if abs_move <= 0:
        sl_pips = DEFAULT_SL_PIPS_FALLBACK
    else:
        sl_raw = abs_move / pip
        sl_pips = float(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return int(sl_pips)

def compute_lot_from_balance(balance, vol, price, risk_pct=0.005, min_lot=0.01, max_lot=1.0):
    risk_amount = balance * risk_pct
    vol = max(vol, 1e-8)
    price_scale = 1000.0
    lot = risk_amount / (vol * price_scale)
    return float(max(min_lot, min(max_lot, round(lot, 2))))

def compute_sl_tp_by_pips(symbol, price, direction, sl_pips, tp_pips):
    pip = pip_value(symbol)
    if direction == "BUY":
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    else:
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    return float(sl), float(tp)

def get_positions_for_symbol(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Cannot get tick for", symbol)
        return {"retcode": None, "comment":"NO_TICK"}
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        return {"retcode": 10009, "price": price, "comment":"DRY_RUN", "direction":direction}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p, "type", None) == 0:  # BUY => close with SELL
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [2]:
def run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets):
    header = not os.path.exists(LOG_FILE)
    acct = mt5.account_info()
    balance = float(acct.balance) if acct else 10000.0

    for safe_name in safe_list:
        raw_symbol = get_mt5_symbol(safe_name)
        print(f"\n--- {raw_symbol} ({safe_name}) ---")
        obs, vol, last_price = fetch_and_build_obs(safe_name, WINDOW, scalers, embeddings, datasets, safe_list)
        if obs is None:
            print("Skip", safe_name)
            continue

        try:
            action, _ = model.predict(obs[np.newaxis,...], deterministic=True)
            a = int(action[0]) if isinstance(action,(list,tuple,np.ndarray)) else int(action)
            print("Signal:", a)
        except Exception as e:
            print("Prediction error:", e)
            continue

        positions = get_positions_for_symbol(raw_symbol)
        print("Existing positions:", len(positions))

        # auto-close opposing
        if a == 1:
            for p in positions:
                if getattr(p,"type",None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p,"type",None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh positions
        positions = get_positions_for_symbol(raw_symbol)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached, skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = compute_lot_from_balance(balance, vol, last_price)
                sl_pips = estimate_sl_pips_from_vol(raw_symbol, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                sl_price, tp_price = compute_sl_tp_by_pips(raw_symbol, last_price, direction, sl_pips, tp_pips)
                res = place_market_order(raw_symbol, direction, lot, sl_price, tp_price)

                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res,"retcode", None)
                    comment = getattr(res,"comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": raw_symbol,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # trailing
        for p in get_positions_for_symbol(raw_symbol):
            # recompute vol-based trail
            trail_pips = max(5, int(estimate_sl_pips_from_vol(raw_symbol, last_price, vol) // 2))
            pip = pip_value(raw_symbol)
            if getattr(p,"type",None) == 0:  # BUY
                cur = mt5.symbol_info_tick(raw_symbol).bid
                new_sl = float(cur - trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw_symbol, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))
            else:  # SELL
                cur = mt5.symbol_info_tick(raw_symbol).ask
                new_sl = float(cur + trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw_symbol, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))

    print("\nSingle pass complete.")


In [4]:
def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets


NameError: name 'DATA_DIR' is not defined

In [6]:
datasets = load_datasets(data_dir=DATA_DIR, window=WINDOW) 

NameError: name 'load_datasets' is not defined

In [3]:
safe_list = sorted(datasets.keys())
print("Assets:", safe_list)
run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)


NameError: name 'datasets' is not defined

In [ ]:
# Use with caution. Set DRY_RUN=False only after testing thoroughly.
# try:
#     while True:
#         run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)
#         time.sleep(60)  # wait one minute (for M1)
# except KeyboardInterrupt:
#     print("Stopped by user")


In [ ]:
if os.path.exists(LOG_FILE):
    df = pd.read_csv(LOG_FILE)
    display(df.tail(30))
else:
    print("No logs yet at", LOG_FILE)
